In [291]:
import cupy as cp

In [292]:
from dgsp.functions import Q, transition_gpu, observation_gpu

In [293]:
transition = transition_gpu


In [294]:
observation = observation_gpu

In [295]:
dt = 1e-3

In [296]:
particles = cp.random.multivariate_normal(mean = cp.zeros(6), cov=cp.diag(cp.ones(6)), size=1000)

In [297]:
x1 = transition(particles.T, 0).reshape((6, 1000)).T.reshape((1000, 6 )) * dt + particles

In [298]:
x2 = cp.apply_along_axis(lambda x: transition(x, 0), arr=particles, axis=1) * dt + particles

In [299]:
cp.allclose(x1, x2)

array(True)

In [300]:
x1.shape

(1000, 6)

In [301]:
x2.shape

(1000, 6)

In [302]:
import matplotlib.pyplot as plt

In [ ]:
traj = 